## Sim Demo

### Imports

In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from env.imports import *

In [23]:
import importlib
import data

import models
import sim.sim
import sim.sim_utils
from sim.sim_utils import bytes2human, print_system_usage
from sim.sim import Simulation
from sim.sim_run import single_sim_run

importlib.reload(sim.sim)
importlib.reload(sim.sim_run) 

absolute_root_path /scratch/asr655/neuroinformatics/GeneEx2Conn


<module 'sim.sim_run' from '/scratch/asr655/neuroinformatics/GeneEx2Conn/sim/sim_run.py'>

#### Check job specs

In [24]:
print_system_usage()

total = psutil.disk_usage('/').total
print(bytes2human(total))

CPU Usage: 33.5%
RAM Usage: 5.4%
Available RAM: 1.4T
Total RAM: 1.5T
52.4G


In [25]:
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

# Check available GPUs
print(f"Available GPUs: {torch.cuda.device_count()}")

for i in range(torch.cuda.device_count()):
    print(f"GPU {i}: {torch.cuda.get_device_name(i)} - Memory Allocated: {torch.cuda.memory_allocated(i)/1024**3:.2f} GB")


0
Available GPUs: 1
GPU 0: NVIDIA H100 80GB HBM3 - Memory Allocated: 1.62 GB


In [26]:
print("XGBoost version:", xgboost.__version__)
print("cupy version:", cp.__version__)

GPUtil.getAvailable()

# if a number is seen a GPU is available
GPUtil.getGPUs()

DEVICE_ID_LIST = GPUtil.getFirstAvailable()
DEVICE_ID = DEVICE_ID_LIST[0] # grab first element from list
if DEVICE_ID != None: 
    print('GPU found', DEVICE_ID)
    use_gpu = True

    GPUtil.showUtilization()

XGBoost version: 2.0.3
cupy version: 13.1.0
GPU found 0
| ID | GPU | MEM |
------------------
|  0 |  0% |  4% |


In [27]:
torch.cuda.is_available()

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

#### Simulation tests <a id="sims"></a>

**Model Parameter Count**
- PCA Bilinear : 730 (27 PCs)
- PLS Bilinear : 101 (10 PLS components)
- Bilinear Lowrank : 73,800 (rank 10)
- PLS MLP : 158,993 (10 PLS components, including PLS projection matrices)
- PCA MLP : 47,873 (27 PCs)
(2 layer)
- MLP : 3,812,609
- SMT : 1,399,947
- MLP w/ CLS : 3,814,145
- SMT w/ CLS : 1,405,579
(3 layer)
- MLP : 7,723,777
- SMT : 2,162,315
- MLP w/ CLS : 7,726,849
- SMT w/ CLS : 2,173,067

**Coord MLP Parameter Count** 
- [32]: 321
- [64, 32]: 2,753
- [128, 64]: 9,601
- [256, 128]: 35,685
- [512, 256, 128]: 169,729

In [30]:
single_sim_run(
              feature_type=[{'transcriptome': None}],
              use_shared_regions=False,
              test_shared_regions=False,
              omit_subcortical=False,
              parcellation='S400',
              hemisphere='both',
              connectome_target='FC',
              binarize=False,
              impute_strategy='mirror_interpolate',
              sort_genes='refgenome',
              gene_list='0.2',
              cv_type='spatial',
              random_seed=9,
              search_method=('wandb', 'mse', 5),
              track_wandb=False,
              skip_cv=True,
              model_type='shared_transformer',
              use_gpu=True, 
              null_model='none', 
              use_folds=[0]
              )

# Clear GPU memory
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Clear CPU memory
gc.collect()

Number of components for 95% variance PCA: 27
Number of 1s: 36662, Number of 0s: 171274, Class balance (1s): 0.176
Number of 1s: 17738, Number of 0s: 190198, Class balance (1s): 0.085
X shape: (455, 7380)
X_pca shape: (455, 27)
Y_sc shape: (455, 455)
Y_sc_spectralL shape: (455, 455)
Y_sc_spectralA shape: (455, 456)
Y_fc shape: (455, 455)
Coordinates shape: (455, 3)
Y shape (455, 455)
Network coverage: 100.0% of regions
Network sizes: [113, 113, 113, 116]
feature_name:  transcriptome
processing_type:  None
features ['transcriptome']
X generated... expanding to pairwise dataset


wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. Additional properties are not allowed ('best_parameters' was unexpected)


Create sweep with ID: mullt3eg
Sweep URL: https://wandb.ai/alexander-ratzan-new-york-university/gx2conn/sweeps/mullt3eg
Initialized sweep with ID: mullt3eg
2
3
4
BEST CONFIG {'input_dim': 14760, 'binarize': False, 'token_encoder_dim': 60, 'd_model': 128, 'encoder_output_dim': 10, 'use_alibi': True, 'nhead': 2, 'num_layers': 2, 'deep_hidden_dims': [8192, 4096, 2048, 1024, 512, 256, 128], 'transformer_dropout': 0.2, 'dropout_rate': 0.2, 'learning_rate': 9e-05, 'weight_decay': 0.0001, 'batch_size': 512, 'aug_prob': 0.3, 'epochs': 70}


  warnings.warn(



Number of learnable parameters in SMT model: 65302155
Using device: cuda
GPU 0: NVIDIA H100 80GB HBM3 - Memory Allocated: 3.44 GB


W0602 20:57:05.943000 22681688094528 torch/_functorch/_aot_autograd/jit_compile_runtime_wrappers.py:551] [0/17] failed to eagerly compile backwards for dynamic, suppressing in case backwards not needed
W0602 20:57:05.943000 22681688094528 torch/_functorch/_aot_autograd/jit_compile_runtime_wrappers.py:551] [0/17] Traceback (most recent call last):
W0602 20:57:05.943000 22681688094528 torch/_functorch/_aot_autograd/jit_compile_runtime_wrappers.py:551] [0/17]   File "/ext3/miniconda3/lib/python3.11/site-packages/torch/_functorch/_aot_autograd/jit_compile_runtime_wrappers.py", line 547, in aot_dispatch_autograd
W0602 20:57:05.943000 22681688094528 torch/_functorch/_aot_autograd/jit_compile_runtime_wrappers.py:551] [0/17]     compiled_bw_func = aot_config.bw_compiler(
W0602 20:57:05.943000 22681688094528 torch/_functorch/_aot_autograd/jit_compile_runtime_wrappers.py:551] [0/17]                        ^^^^^^^^^^^^^^^^^^^^^^^
W0602 20:57:05.943000 22681688094528 torch/_functorch/_aot_autograd

TypeError: TracerBase.create_proxy() missing 1 required positional argument: 'kwargs'